Steps of Data Preprocessing
1. Import libraries
2. Read data
3. Checking for missing values
4. Checking for categorical data
5. Standardize the data
6. PCA transformation
7. Data splitting

1. Import libraries

In [1]:
# main libraries
import pandas as pd
import numpy as np
import time

# visual libraries
from matplotlib import pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns

2. Read data

In [2]:
telco_cust = pd.read_excel('telco_new.xlsx')
telco_cust.head()

,region,tenure,age,marital,address,income,ed,employ,retire,gender,...,confer,ebill,loglong,logtoll,logequi,logcard,logwire,lninc,custcat,churn
0,Zone 2,13,44,Married,9,64,College degree,5,No,Male,...,No,No,1.308333,#NULL!,#NULL!,2.014903,#NULL!,4.158883,Basic service,Yes
1,Zone 3,11,33,Married,7,136,Post-undergraduate degree,5,No,Male,...,Yes,No,1.481605,3.032546,#NULL!,2.72458,3.575151,4.912655,Total service,Yes
2,Zone 3,68,52,Married,24,116,Did not complete high school,29,No,Female,...,Yes,No,2.898671,2.890372,#NULL!,3.409496,#NULL!,4.753590,Plus service,No
3,Zone 2,33,33,Unmarried,12,33,High school degree,0,No,Female,...,No,No,2.246015,#NULL!,#NULL!,#NULL!,#NULL!,3.496508,Basic service,Yes
4,Zone 2,23,30,Married,9,30,Did not complete high school,2,No,Male,...,Yes,No,1.840550,#NULL!,#NULL!,#NULL!,#NULL!,3.401197,Plus service,No


In [ ]:
telco_cust.shape

In [ ]:
telco_cust.info()

In [ ]:
telco_cust.describe(include = 'all')

In [ ]:
print(telco_cust.columns.values)

In [ ]:
telco_cust.dtypes

4. Checking for categorical data

Which features are categorical?

Categorical features: region, marital, ed, retire, gender, tollfree, equip, callcard, wireless, multline, voice, pager, internet, callid, callwait, forward, confer, ebil, custcat, churn

Which features are numerical?

Numerical feature: tenure, age, address, income, employ, reside, longmon, tollmon, equipmon, cardmon, wiremon, longten, tollten, equipten, cardten, wireten, loglong, logtoll, logequi, logcard, logwire, lninc

logtoll, logequi, logcard, logwire should be converted to numerical data type (they are object at the moment)  

In [3]:
object_to_numeric = ['logtoll', 'logequi', 'logcard', 'logwire']

for item in object_to_numeric:
    telco_cust[item] = pd.to_numeric(telco_cust[item], errors='coerce')
    
telco_cust.dtypes

region       object
tenure        int64
age           int64
marital      object
address       int64
income        int64
ed           object
employ        int64
retire       object
gender       object
reside        int64
tollfree     object
equip        object
callcard     object
wireless     object
longmon     float64
tollmon     float64
equipmon    float64
cardmon     float64
wiremon     float64
longten     float64
tollten     float64
equipten    float64
cardten     float64
wireten     float64
multline     object
voice        object
pager        object
internet     object
callid       object
callwait     object
forward      object
confer       object
ebill        object
loglong     float64
logtoll     float64
logequi     float64
logcard     float64
logwire     float64
lninc       float64
custcat      object
churn        object
dtype: object

3. Checking for missing values

In [ ]:
telco_cust.isnull().sum()

Out of the 1000 datapoints, too many missing values on these 4 fields (logtoll, logequi, logcard, logwire), I'm going to remove these columns completely

In [4]:
columns_to_drop = ['logtoll', 'logequi', 'logcard', 'logwire']
    
telco_cust.drop(columns_to_drop, axis =1, inplace = True)
telco_cust.isnull().sum()

region      0
tenure      0
age         0
marital     0
address     0
income      0
ed          0
employ      0
retire      0
gender      0
reside      0
tollfree    0
equip       0
callcard    0
wireless    0
longmon     0
tollmon     0
equipmon    0
cardmon     0
wiremon     0
longten     0
tollten     0
equipten    0
cardten     0
wireten     0
multline    0
voice       0
pager       0
internet    0
callid      0
callwait    0
forward     0
confer      0
ebill       0
loglong     0
lninc       0
custcat     0
churn       0
dtype: int64

# EDA

Demographics - To understand the gender, age range, patner and dependent status of the customers

In [ ]:
# Gender Distribution

colors = ['#4D3425','#E4512B']
ax = (telco_cust['gender'].value_counts()*100.0 /len(telco_cust)).plot(kind='bar',
                                                                           stacked = True,
                                                                          rot = 0,
                                                                          color = colors)
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_ylabel('% Customers')
ax.set_xlabel('Gender')
ax.set_ylabel('% Customers')
ax.set_title('Gender Distribution')

# create a list to collect the plt.patches data
totals = []

# find the values and append to list
for i in ax.patches:
    totals.append(i.get_width())

# set individual bar lables using above list
total = sum(totals)

for i in ax.patches:
    # get_width pulls left or right; get_y pushes up or down
    ax.text(i.get_x()+.15, i.get_height()-3.5, \
            str(round((i.get_height()/total), 1))+'%',
            fontsize=12,
            color='white',
           weight = 'bold')

In [ ]:
# Retirement rate : There are only 4.7% of the customers who are retired. Thus most of our customers in the data are not retired people.

ax = (telco_cust['retire'].value_counts()*100.0 /len(telco_cust))\
.plot.pie(autopct='%.1f%%', labels = ['No', 'Yes'],figsize =(5,5), fontsize = 12 )                                                                           
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_ylabel('Senior Citizens',fontsize = 12)
ax.set_title('% of Senior Citizens', fontsize = 12)

In [ ]:
# Partner and dependent status: half of the customers are married

ax = (telco_cust['marital'].value_counts()*100.0 /len(telco_cust))\
.plot.pie(autopct='%.1f%%', labels = ['Unmarried', 'Married'],figsize =(5,5), fontsize = 12 )                                                                           
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_title('% of Married people', fontsize = 12)

Based on the figure above: most positive effect to churn: equip_Yes, internet_Yes, ebill_Yes, callcard_No, most negative effect: tenure, loglong, equip_No, longten

Customer Account Information: checking tenure (Number of years a customer is using the service), custcat (service category the customer is using)

In [ ]:
# Tenure

ax = sns.distplot(telco_cust['tenure'], hist=True, kde=False, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
ax.set_ylabel('# of Customers')
ax.set_xlabel('Tenure')
ax.set_title('# of Customers by their tenure')

In [ ]:
# Custcat

ax = telco_cust['custcat'].value_counts().plot(kind = 'bar',rot = 0, width = 0.3)
ax.set_ylabel('# of Customers')
ax.set_title('# of Customers by Service Type')

In [ ]:
# Checking the tenure of customers based on their service type

fig, (ax1,ax2,ax3, ax4) = plt.subplots(nrows=1, ncols=4, sharey = True, figsize = (20,6))

ax = sns.distplot(telco_cust[telco_cust['custcat']=='Basic service']['tenure'],
                   hist=True, kde=False,
                   bins=int(180/5), color = 'turquoise',
                   hist_kws={'edgecolor':'black'},
                   kde_kws={'linewidth': 4},
                 ax=ax1)
ax.set_ylabel('# of Customers')
ax.set_xlabel('Tenure')
ax.set_title('Basic service')

ax = sns.distplot(telco_cust[telco_cust['custcat']=='E-service']['tenure'],
                   hist=True, kde=False,
                   bins=int(180/5), color = 'red',
                   hist_kws={'edgecolor':'black'},
                   kde_kws={'linewidth': 4},
                 ax=ax2)
ax.set_xlabel('Tenure')
ax.set_title('E-service')

ax = sns.distplot(telco_cust[telco_cust['custcat']=='Total service']['tenure'],
                   hist=True, kde=False,
                   bins=int(180/5), color = 'steelblue',
                   hist_kws={'edgecolor':'black'},
                   kde_kws={'linewidth': 4},
                 ax=ax3)
ax.set_xlabel('Tenure',size = 14)
ax.set_title('Total service',size = 14)

ax = sns.distplot(telco_cust[telco_cust['custcat']=='Plus service']['tenure'],
                   hist=True, kde=False,
                   bins=int(180/5), color = 'darkblue',
                   hist_kws={'edgecolor':'black'},
                   kde_kws={'linewidth': 4},
                 ax=ax4)

ax.set_xlabel('Tenure')
ax.set_title('Plus service')

Most of the basic service contracts last for a few months, while the contracts with extras tend to last much longer than that. This shows that the customers paying more for a service are more loyal to the company and tend to stay with it for a longer period of time

Analyzing our dependent feature - churn

In [ ]:
#The dataset is labeled as 'Yes' and 'No' for the Churn column

all = telco_cust.shape[0]
churn = telco_cust[telco_cust['churn'] == 'Yes']
no_churn = telco_cust[telco_cust['churn'] == 'No']

x = len(churn)/all
y = len(no_churn)/all

print('churn :',x*100,'%')
print('no churn :',y*100,'%')

In [ ]:
# Let's plot the Churn class against the Frequency
labels = ['No churn','Churn']
classes = pd.value_counts(telco_cust['churn'], sort = True)
classes.plot(kind = 'bar', rot=0)
plt.title("Churn class distribution")
plt.xticks(range(2), labels)
plt.xlabel("Churn")
plt.ylabel("Frequency")

Churn vs Tenure: As we saw, the customers who do not churn, they tend to stay for a longer tenure with the telecom company

In [ ]:
sns.boxplot(x = telco_cust.churn, y = telco_cust.tenure)

Churn by Service Type

In [ ]:
colors = ['#4D3425','#E4512B']
contract_churn = telco_cust.groupby(['custcat','churn']).size().unstack()

ax = (contract_churn.T*100.0 / contract_churn.T.sum()).T.plot(kind='bar',
                                                                width = 0.3,
                                                                stacked = True,
                                                                rot = 0, 
                                                                figsize = (10,6),
                                                                color = colors)
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.legend(loc='best',prop={'size':14},title = 'Churn')
ax.set_ylabel('% Customers',size = 14)
ax.set_title('Churn by Service Type',size = 14)

# Code to add the data labels on the stacked bar chart
for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.annotate('{:.0f}%'.format(height), (p.get_x()+.25*width, p.get_y()+.4*height),
                color = 'white',
               weight = 'bold',
               size = 14)

Churn by Retirement

In [ ]:
colors = ['#4D3425','#E4512B']
seniority_churn = telco_cust.groupby(['retire','churn']).size().unstack()

ax = (seniority_churn.T*100.0 / seniority_churn.T.sum()).T.plot(kind='bar',
                                                                width = 0.2,
                                                                stacked = True,
                                                                rot = 0, 
                                                                figsize = (8,6),
                                                                color = colors)
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.legend(loc='center',prop={'size':14},title = 'Churn')
ax.set_ylabel('% Customers')
ax.set_title('Churn by Seniority Level',size = 14)

# Code to add the data labels on the stacked bar chart
for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.annotate('{:.0f}%'.format(height), (p.get_x()+.25*width, p.get_y()+.4*height),
                color = 'white',
               weight = 'bold',size =14)

Based on this figure, non-retired citizens are more likely to churn

Churn by Monthly Charges

In [ ]:
ax = sns.kdeplot(telco_cust.longmon[(telco_cust["churn"] == 'No')],
                color="Red", shade = True)
ax = sns.kdeplot(telco_cust.longmon[(telco_cust["churn"] == 'Yes')],
                ax =ax, color="Blue", shade= True)
ax.legend(["Not Churn","Churn"],loc='upper right')
ax.set_ylabel('Density')
ax.set_xlabel('Monthly Charges')
ax.set_title('Distribution of monthly charges by churn')

Churn by Total Charges

In [ ]:
ax = sns.kdeplot(telco_cust.loglong[(telco_cust["churn"] == 'No') ],
                color="Red", shade = True)
ax = sns.kdeplot(telco_cust.loglong[(telco_cust["churn"] == 'Yes') ],
                ax =ax, color="Blue", shade= True)
ax.legend(["Not Churn","Churn"],loc='upper right')
ax.set_ylabel('Density')
ax.set_xlabel('Total Bill')
ax.set_title('Distribution of total charges by churn')

Data preprocessing

Standardize the data

In [5]:
#Converting the predictor variable in a binary numeric variable

telco_cust['churn'].replace(to_replace='Yes', value=1, inplace=True)
telco_cust['churn'].replace(to_replace='No',  value=0, inplace=True)

In [6]:
#Let's convert all the categorical variables into dummy variables

df_dummies = pd.get_dummies(telco_cust)
df_dummies.head()

,tenure,age,address,income,employ,reside,longmon,tollmon,equipmon,cardmon,...,forward_No,forward_Yes,confer_No,confer_Yes,ebill_No,ebill_Yes,custcat_Basic service,custcat_E-service,custcat_Plus service,custcat_Total service
0,13,44,9,64,5,2,3.70,0.00,0.0,7.50,...,0,1,1,0,1,0,1,0,0,0
1,11,33,7,136,5,6,4.40,20.75,0.0,15.25,...,0,1,0,1,1,0,0,0,0,1
2,68,52,24,116,29,2,18.15,18.00,0.0,30.25,...,1,0,0,1,1,0,0,0,1,0
3,33,33,12,33,0,1,9.45,0.00,0.0,0.00,...,1,0,1,0,1,0,1,0,0,0
4,23,30,9,30,2,4,6.30,0.00,0.0,0.00,...,0,1,0,1,1,0,0,0,1,0


In [ ]:
# heat map of correlation of features
correlation_matrix = df_dummies.corr()
fig = plt.figure(figsize=(12,9))
sns.heatmap(correlation_matrix,vmax=0.8,square = True)
plt.show()

In [ ]:
#Get Correlation of "Churn" with other variables:

plt.figure(figsize=(15,8))
df_dummies.corr()['churn'].sort_values(ascending = False).plot(kind='bar')

- positive correlation: equip_Yes, internet_Yes, ebill_Yes, equipmon, callcard_No
- negative correlation: cardten, address, callcard_Yes, age, ebill_No, internet_No, longmon, employ, longten, equip_No, loglong, tenure

In [ ]:
dep_var = ["churn"]
indep_var = ["equip_Yes", "internet_Yes", "ebill_Yes", "equipmon", "callcard_No", "cardten", "callcard_Yes", "age", "ebill_No", "internet_No", "longmon", "employ", "longten", "equip_No", "loglong", "tenure"]

In [7]:
# deciding to drop some columns?

dep_var = ["churn"]
indep_var = ["equip_Yes", "internet_Yes", "ebill_Yes", "equipmon", "callcard_No", "cardten", "callcard_Yes", "age", "ebill_No", "internet_No", "longmon", "employ", "longten", "equip_No", "loglong", "tenure"]

df_dummies_selected = df_dummies[dep_var+indep_var]
df_dummies_selected.head()


,churn,equip_Yes,internet_Yes,ebill_Yes,equipmon,callcard_No,cardten,callcard_Yes,age,ebill_No,internet_No,longmon,employ,longten,equip_No,loglong,tenure
0,1,0,0,0,0.0,0,110.0,1,44,1,1,3.70,5,37.45,1,1.308333,13
1,1,0,0,0,0.0,0,125.0,1,33,1,1,4.40,5,42.00,1,1.481605,11
2,0,0,0,0,0.0,0,2150.0,1,52,1,1,18.15,29,1300.60,1,2.898671,68
3,1,0,0,0,0.0,1,0.0,0,33,1,1,9.45,0,288.80,1,2.246015,33
4,0,0,0,0,0.0,1,0.0,0,30,1,1,6.30,2,157.05,1,1.840550,23


Import machine learning libraries

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler

In [9]:
def data_prep(df):
    y = df['churn'].values
    X = df.drop(columns = ['churn'])

    features = X.columns.values
    scaler = StandardScaler()
    scaler.fit(X)
    X = pd.DataFrame(scaler.transform(X))
    X.columns = features
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=105)
    return X_train, X_test, y_train, y_test

Logistic Regression

In [10]:
# on all column

X_train, X_test, y_train, y_test = data_prep(df_dummies)

model_regression = LogisticRegression()
result = model_regression.fit(X_train, y_train)

prediction_train = model_regression.predict(X_train)
prediction_test = model_regression.predict(X_test)

print("Accuracy score on all columns:", accuracy_score(y_test, prediction_test))

# on selected dataset

X_train, X_test, y_train, y_test = data_prep(df_dummies_selected)

model = LogisticRegression()
model_regression.fit(X_train, y_train)

prediction_train = model_regression.predict(X_train)
prediction_test = model_regression.predict(X_test)

print("Accuracy score on selected dataset:", accuracy_score(y_test, prediction_test))

Accuracy score on all columns: 0.805
Accuracy score on selected dataset: 0.805


In [27]:
# Other evaulation scores 

# confusion matrix: a summarized table used to assess the performance of a classification model. The number of correct and incorrect predictions are summarized with count values and broken down by each class

print('Confusion matrix on selected dataset:\n', confusion_matrix(y_test, prediction_test))

# precision: attempts to answer “What proportion of positive identifications was actually correct?”

print('Precision on selected dataset:\n', precision_score(y_test, prediction_test))

# recall: attempts to answer “What proportion of actual positives was identified correctly?”

print('Recall on selected dataset:\n', recall_score(y_test, prediction_test))

Confusion matrix on selected dataset:
 [[93 58]
 [ 6 43]]
Precision on selected dataset:
 0.42574257425742573
Recall on selected dataset:
 0.8775510204081632


Random Forest

In [17]:
# on all column

X_train, X_test, y_train, y_test = data_prep(df_dummies)

model_rf = RandomForestClassifier(n_estimators=1000 , oob_score = True, n_jobs = -1,
                                  random_state =50, max_features = "auto",
                                  max_leaf_nodes = 30)
model_rf.fit(X_train, y_train)

prediction_test = model_rf.predict(X_test)
print("Accuracy score on all columns:", accuracy_score(y_test, prediction_test))

# on selected dataset

X_train, X_test, y_train, y_test = data_prep(df_dummies_selected)

model_rf = RandomForestClassifier(n_estimators=1000 , oob_score = True, n_jobs = -1,
                                  random_state =50, max_features = "auto",
                                  max_leaf_nodes = 30)
model_rf.fit(X_train, y_train)

prediction_test = model_rf.predict(X_test)
print("Accuracy score on selected dataset:", accuracy_score(y_test, prediction_test))

Accuracy score on all columns: 0.805
Accuracy score on selected dataset: 0.815


In [18]:
# confusion matrix

print('Confusion matrix on selected dataset:\n', confusion_matrix(y_test, prediction_test))

# precision

print('Precision on selected dataset:\n', precision_score(y_test, prediction_test))

# recall

print('Recall on selected dataset:\n', recall_score(y_test, prediction_test))

Confusion matrix on selected dataset:
 [[137  14]
 [ 23  26]]
Precision on selected dataset:
 0.65
Recall on selected dataset:
 0.5306122448979592


Support Vecor Machine (SVM)

In [20]:
# on all column

X_train, X_test, y_train, y_test = data_prep(df_dummies)

model_svm = SVC(kernel='linear') 
model_svm.fit(X_train,y_train)
prediction_test = model_svm.predict(X_test)

print("Accuracy score on all columns:", accuracy_score(y_test, prediction_test))

# on selected dataset

X_train, X_test, y_train, y_test = data_prep(df_dummies_selected)

model_svm = SVC(kernel='linear') 
model_svm.fit(X_train,y_train)
prediction_test = model_svm.predict(X_test)
print("Accuracy score on selected dataset:", accuracy_score(y_test, prediction_test))

Accuracy score on all columns: 0.79
Accuracy score on selected dataset: 0.81


In [21]:
# confusion matrix

print('Confusion matrix on selected dataset:\n', confusion_matrix(y_test, prediction_test))

# precision

print('Precision on selected dataset:\n', precision_score(y_test, prediction_test))

# recall

print('Recall on selected dataset:\n', recall_score(y_test, prediction_test))

Confusion matrix on selected dataset:
 [[137  14]
 [ 24  25]]
Precision on selected dataset:
 0.6410256410256411
Recall on selected dataset:
 0.5102040816326531


Gaussian Naive Bayes

In [23]:
# on all column

X_train, X_test, y_train, y_test = data_prep(df_dummies)

model_gnb = GaussianNB()
model_gnb.fit(X_train, y_train)
prediction_test = model_gnb.predict(X_test)

print("Accuracy score on all columns:", accuracy_score(y_test, prediction_test))

# on selected dataset

X_train, X_test, y_train, y_test = data_prep(df_dummies_selected)

model_gnb = GaussianNB()
model_gnb.fit(X_train, y_train)
prediction_test = model_gnb.predict(X_test)

print("Accuracy score on selected dataset:", accuracy_score(y_test, prediction_test))

Accuracy score on all columns: 0.64
Accuracy score on selected dataset: 0.68


Hyperparameter Tuning to improve Accuracy
- Var_smoothing (Variance smoothing) parameter specifies the portion of the largest variance of all features to be added to variances for stability of calculation.

- Gaussian Naive Bayes assumes that features follows normal distribution which is most unlikely in real world.So solve this problem we can perform "power transformation" on each feature to make it more or less normally distributed. By default, PowerTransformer results in features that have a 0 mean and 1 standard deviation.

In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import PowerTransformer

X_train, X_test, y_train, y_test = data_prep(df_dummies_selected)

model_gnb_tuned = GaussianNB()
np.logspace(0,-9, num=10)

cv_method = RepeatedStratifiedKFold(n_splits=5, 
                                    n_repeats=3, 
                                    random_state=999)

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

gs_NB = GridSearchCV(estimator=model_gnb_tuned, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='accuracy')

Data_transformed = PowerTransformer().fit_transform(X_test)

gs_NB.fit(Data_transformed, y_test)

results_NB = pd.DataFrame(gs_NB.cv_results_['params'])
results_NB['test_score'] = gs_NB.cv_results_['mean_test_score']

predict_test = gs_NB.predict(Data_transformed)
accuracy_test = accuracy_score(y_test,predict_test)
print('accuracy_score on test dataset : ', accuracy_test)

Fitting 15 folds for each of 100 candidates, totalling 1500 fits
accuracy_score on test dataset :  0.785


In [26]:
# confusion matrix

print('Confusion matrix on selected dataset:\n', confusion_matrix(y_test, prediction_test))

# precision

print('Precision on selected dataset:\n', precision_score(y_test, prediction_test))

# recall

print('Recall on selected dataset:\n', recall_score(y_test, prediction_test))

Confusion matrix on selected dataset:
 [[93 58]
 [ 6 43]]
Precision on selected dataset:
 0.42574257425742573
Recall on selected dataset:
 0.8775510204081632


In [ ]:
#saving the 4 models for FastAPI

import pickle

In [ ]:
filename = 'churn_model_regression.sav'
pickle.dump(model_regression, open(filename, 'wb'))

filename = 'churn_model_rf.sav'
pickle.dump(model_rf, open(filename, 'wb'))

filename = 'churn_model_svm.sav'
pickle.dump(model_svm, open(filename, 'wb'))

filename = 'churn_model_gnb_tuned.sav'
pickle.dump(model_gnb_tuned, open(filename, 'wb'))